In [ ]:
import os, json, time, math
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv
from tqdm import tqdm
from openai import OpenAI

INPUT_JSON_PATH  = "../eventic_graph.json"                 # 원본 데이터 경로
OUTPUT_JSON_PATH = "../eventic_with_embedding.json" # 저장 경로

with open(INPUT_JSON_PATH, "r", encoding="utf-8") as f:
    data: List[Dict[str, Any]] = json.load(f)

print(f"로드된 항목 수: {len(data)}")
load_dotenv()  # .env 파일에서 OPENAI_API_KEY 읽기
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("환경변수 OPENAI_API_KEY가 설정되지 않았습니다. .env 파일을 확인하세요.")
    


로드된 항목 수: 14


In [5]:
client = OpenAI(api_key=api_key)

EMBED_MODEL = "text-embedding-3-large"  # 최신 임베딩 모델
MAX_RETRIES = 5                         # 재시도 횟수
BASE_BACKOFF = 1.5                      # 지수 백오프 배수

def get_embedding(text: str, model: str = EMBED_MODEL) -> List[float]:
    """
    OpenAI Embeddings API 호출로 텍스트 임베딩을 반환합니다.
    네트워크/한도 오류 등에 대해 지수 백오프 재시도를 수행합니다.
    """
    if text is None:
        text = ""
    text = text.strip()

    last_err = None
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            resp = client.embeddings.create(model=model, input=text)
            emb = resp.data[0].embedding
            return emb
        except Exception as e:
            last_err = e
            sleep_s = (BASE_BACKOFF ** attempt) + (0.1 * attempt)
            print(f"[경고] 임베딩 실패 (시도 {attempt}/{MAX_RETRIES}) -> {e}\n{sleep_s:.1f}s 대기 후 재시도")
            time.sleep(sleep_s)
    # 모두 실패한 경우
    raise RuntimeError(f"임베딩 생성에 반복 실패: {last_err}")

In [8]:
updated = 0
skipped = 0

for item in tqdm(data, desc="Embedding Conditions"):
    # 이미 embedding이 있으면 건너뛰고 싶다면 아래 조건 사용
    if "embedding" in item and isinstance(item["embedding"], list) and item["embedding"]:
        skipped += 1
        continue

    condition_text = item.get("Condition", "")
    emb = get_embedding(condition_text, model=EMBED_MODEL)
    item["embedding"] = emb
    item["embedding_model"] = EMBED_MODEL
    item["embedding_dim"] = len(emb)
    updated += 1

print(f"업데이트 완료: {updated}개, 스킵: {skipped}개")
# 결과 저장
with open(OUTPUT_JSON_PATH, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print(f"저장 완료: {OUTPUT_JSON_PATH}")


Embedding Conditions: 100%|██████████| 14/14 [00:00<00:00, 155756.65it/s]

업데이트 완료: 0개, 스킵: 14개
저장 완료: ../eventic_with_embedding.json
